In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
pip install "ipython-beautifulsoup[bs4]"

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup

In [5]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [6]:
ListLeagues=[]
ListOfTeams=[]
ListMarketValue=[]
ListOfLinks=[]
transfermarkt="https://www.transfermarkt.com"
for i in range(1,5):
    testpage="https://www.transfermarkt.com/vereins-statistik/wertvollstemannschaften/marktwertetop/plus/land_id/157/kontinent_id/6/yt0/Show/0//page/"+str(i)
    pageTree= requests.get(testpage,headers=headers)
    pageSoup=BeautifulSoup(pageTree.content,'html.parser')
    teams=pageSoup.find_all("td", {"class": "no-border-links hauptlink"})
    league=pageSoup.find_all("td", {"class": "links"})
    values=pageSoup.find_all("td", {"class": "rechts"})
    links=pageSoup.select("a.vereinprofil_tooltip")
    for j in range(0,25):
        ListLeagues.append(league[j].text)
        ListOfTeams.append(teams[j].text)
        ListMarketValue.append(values[j].text)
    for j in range(0,50,2):
        ListOfLinks.append(transfermarkt+links[j].get("href"))
        

In [7]:
df=pd.DataFrame({"Team":ListOfTeams,"League":ListLeagues,"Value":ListMarketValue,"Link":ListOfLinks})

In [8]:
df.to_csv("data_csv/laliga_market_values.csv")

In [9]:
page="https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id=2020"
pageTree= requests.get(page,headers=headers)
pageSoup=BeautifulSoup(pageTree.content,'html.parser')
table=pageSoup.find('div' ,{"class":"responsive-table"})

In [10]:
for i in range(2020,2008,-1):
    page="https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id="+str(i)
    pageTree= requests.get(page,headers=headers)
    pageSoup=BeautifulSoup(pageTree.content,'html.parser')
    table=pageSoup.find('div' ,{"class":"responsive-table"})
    ListPlace=[]
    ListName=[]
    ListMP=[]
    ListW=[]
    ListD=[]
    ListL=[]
    ListG=[]
    ListGD=[]
    ListP=[]
    for team in table.find_all('tbody'):
        rows=team.find_all('tr')
        for row in rows:
            place=name=row.find('td',{"class":"rechts hauptlink"}).text.strip()
            name=row.find('td',{"class":"no-border-links hauptlink"}).text.strip()
            stats= row.find_all('td',{"class":"zentriert"})
            ListPlace.append(place)
            ListName.append(name)
            ListMP.append(stats[1].text)
            ListW.append(stats[2].text)
            ListD.append(stats[3].text)
            ListL.append(stats[4].text)
            ListG.append(stats[5].text)
            ListGD.append(stats[6].text)
            ListP.append(stats[7].text)
    Table=pd.DataFrame({"Team":ListName,"Match Played":ListMP,"Won":ListW,"Loose":ListL,"Goals":ListG,"Goal Difference":ListGD,"Points":ListP},index=ListPlace)
    tablepath="data_csv/standings"+"-"+str(i+1)+"-"+str(i)+".csv"
    Table.to_csv(tablepath)

In [11]:
Table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 1 to 20
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Team             20 non-null     object
 1   Match Played     20 non-null     object
 2   Won              20 non-null     object
 3   Loose            20 non-null     object
 4   Goals            20 non-null     object
 5   Goal Difference  20 non-null     object
 6   Points           20 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB
